### tfidf

In [7]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [34]:
def create_tfidfvector(vocab=None,max_feature=500):
    if vocab:
        return TfidfVectorizer(stopwords,
                                           smooth_idf=True,
                                           token_pattern=r"(?u)\b\w+\b",
                                           max_features=max_feature,
                                           vocabulary=vocab 
                                          )
    else:
        return TfidfVectorizer(stopwords,
#                                            smooth_idf=True,
                                           token_pattern=r"(?u)\b\w+\b",
                                           max_features=max_feature,
                                          )
tfidf_vectorizer = create_tfidfvector()
texts = ["我今 北京 下雨了", "我今 加班"]
tfidf_vectorizer.fit_transform(texts)                             

<2x4 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [24]:
tfidf_vectorizer.vocabulary_  # 词表：词对应的index

{'我今': 3, '北京': 2, '下雨了': 0, '加班': 1}

In [31]:
tfidf_vectorizer.get_feature_names()

['下雨了', '加班', '北京', '我今']

In [29]:
# 
tfidf_vectorizer.transform([texts[0]]).toarray()  # 

array([[0.6316672 , 0.        , 0.6316672 , 0.44943642]])

In [30]:
tfidf_vectorizer.transform([texts[1]]).toarray()

array([[0.        , 0.81480247, 0.        , 0.57973867]])

In [38]:
# CountVectorizer?
def create_countvector(vocab,max_feature=500):
        return CountVectorizer(stopwords,
                                vocabulary=vocab,
                               token_pattern=r"(?u)\b\w+\b",
                               max_features=max_feature,
                                      )

### 字/词向量

In [6]:
import codecs
import numpy as np

In [7]:
def load_pretrained_vec(pretrained_path,vec_len):
    pre_trained = {}
    for i, line in enumerate(codecs.open(pretrained_path, 'r', 'utf-8')):
        line = line.rstrip().split()
        # print(line)
        if len(line) == vec_len + 1:
            pre_trained[line[0]] = np.array(
                [float(x) for x in line[1:]]
            ).astype(np.float32)
    return pre_trained

In [8]:
char_pre_trained = load_pretrained_vec("/opt/project/Jupyter/StaticFile/char_wiki_100.utf8",vec_len=100)

In [23]:
# word_pre_trained = load_pretrained_vec("/opt/project/Jupyter/StaticFile/sgns.financial.word",vec_len=300)

In [26]:
# char_pre_trained['一']

### 短语/句子/段落的向量化

In [16]:
def resize_weight(keywords):
    """
    归一化权重
    """
#     print(type(keywords))
    a = 0.0
    new_ks = []
    for k in keywords:
#         print(k)
        a +=k[1]
    for k in keywords:
        new_ks.append((k[0],k[1]/a))
    return new_ks

def weight_char_embed(chars, top_k=5,vec_len=100):
    """
    短语向量表示：加权词向量
    """
    vec = np.zeros((vec_len))
    chars = chars.replace(" ", "")
    keywords = get_keywords(chars)
    keywords = keywords[0:top_k]
#     print(keywords)
    keywords = resize_weight(keywords)
#     print(keywords)
    for keyword in keywords:
        for char in keyword[0]:
            if char in char_pre_trained.keys():
                vec +=char_pre_trained[char]*keyword[1]
    return vec

def batch_wce(lines):
    return list(map(weight_char_embed,lines))

In [18]:
# chars = "信息检索的PR曲线（Precision-Recall Curve）作图"
# weight_char_embed(chars)